In [52]:
import json
from pathlib import Path
import os
import pprint
class Detectors:
    def __init__(self):
        self.missions = []
        self.sites = []
        self.assets = []
        self.load_config()
        self.load_assets()
        
    
    def load_config(self):
        data = None
        with open('./config.json') as f:
          data = json.load(f)
        self.missions = data["missions"]
        self.sites = data["sites"]
        
    def load_assets(self):
        data = None
        with open('./config.json') as f:
            data = json.load(f)
        self.missions = data["missions"]
        self.sites = data["sites"]

        p = Path('./assets')
        for child in p.iterdir():
            filename, ext = os.path.splitext(str(child))
            if(ext == ".json"):
                with open(str(child)) as f:
                    self.assets.append(json.load(f))
    def print(self):
        pp = pprint.PrettyPrinter(indent=4)
        print("\n--- Missions ---")
        pp.pprint(self.missions)
        print("\n--- Sites ---")
        pp.pprint(self.sites)
        print("\n--- Assets ---")
        pp.pprint(self.assets)
        

a = Detectors()
a.print()


--- Missions ---
[   {   'area': 'North America',
        'description': 'Protecting the homeland from airbourne threats.',
        'id': 0,
        'name': 'Air Defense'},
    {   'area': 'Pacific Ocean, Atlantic Ocean',
        'description': 'Detect and warn about seabourne threats to teh '
                       'homeland.',
        'id': 1,
        'name': 'Maritime Warning'}]

--- Sites ---
[   {'id': 0, 'mission_ids': [0], 'name': 'Buckley Air Force Base'},
    {   'id': 1,
        'mission_ids': [0, 1],
        'name': 'Cheyenne Mountain Air Force Base'}]

--- Assets ---
[   {   'fields': ['power', 'connection', 'fan_speed', 'data_rate'],
        'id': 1,
        'mission_criticalities': [2, 2],
        'mission_ids': [0, 1],
        'name': 'Netgear Nighthawk R6700 - AC1750',
        'site_id': 0,
        'tests': [   {   'id': 0,
                         'if': [],
                         'test_classification': 2,
                         'then_conditions': [   {   'field': 

TypeError: load_config() takes 1 positional argument but 2 were given